In [1]:
from pyspark import SparkConf, SparkContext
import binascii

In [2]:
def mapper_split(s):
    str_split = []
    str_return = []
    str_split = s.split()
    for word in str_split:
        if word != "" and ((ord(word[0]) >= 48 and ord(word[0]) <= 57) or (ord(word[0]) >= 65 and ord(word[0]) <= 90) or (ord(word[0]) >= 97 and ord(word[0]) <= 122)):
            str_return.append(word)
    return (0, str_return)



In [3]:
def mapper_combine3(line):
    shingle = []
    shingle_set = []
    tmp = ""
    for i in range(len(line[1])):
        tmp = line[1][i] + ' ' + line[1][i+1] + ' ' + line[1][i+2]
        tmp = binascii.crc32(tmp.encode('utf8'))
        shingle.append(tmp)
        if i == len(line[1])-3:
            break
    for i in range(len(shingle)):
        if shingle[i] not in shingle_set:
            shingle_set.append(shingle[i])
    return shingle_set

In [4]:
def mapper_into_all_index(line): # to find the index in All_shingle
    index_list = []
    for i in range(1,len(line)):
        index = [All_shingle.index(line[i])]  # Need to turn the index turn to list type to make it an element of list
        index_list.append((line[0], index))
    return index_list

In [5]:
data = [0] * 102
All_shingle = []
Shingle_docs = [0] * 101
All_shingle_len = 0
for i in range(1, 102):
    if i < 10:
        data[i] = sc.textFile("athletics/00"+str(i)+".txt")
    elif i < 100:
        data[i] = sc.textFile("athletics/0"+str(i)+".txt")
    elif i < 102:
        data[i] = sc.textFile("athletics/"+str(i)+".txt")
    data[i] = data[i].map(mapper_split).reduceByKey(lambda a, b: a+b).flatMap(mapper_combine3)
    doc = data[i].collect()
    for word in doc:
        if word not in All_shingle:
            All_shingle.append(word)
    doc.append(i)
    doc.sort()
    Shingle_docs[i-1] = doc
All_shingle.sort()

In [6]:
# change every shingle doc into rdd
Shingle_docs_RDD = sc.parallelize(Shingle_docs)
Shingle_docs_RDD = Shingle_docs_RDD.flatMap(mapper_into_all_index).reduceByKey(lambda a, b: a+b).sortByKey(ascending = True)
#print(Shingle_docs_RDD.collect())

In [7]:
def mapper_hash(line):
    tmp = [All_shingle_len+1] * 100
    real_hash = [All_shingle_len+1] * 100
    for i in range(len(line[1])):
        for j in range(100):
            tmp[j] = (((j+2)*line[1][i] + 3*j ) % All_shingle_len)
            if tmp[j] < real_hash[j]:
                real_hash[j] = tmp[j]
    # return documents as key hash_value of 100 hash function as value(list)
    return line[0], real_hash

In [8]:
# Do the min hash
All_shingle_len = len(All_shingle)
Shingle_docs_RDD = Shingle_docs_RDD.map(mapper_hash)
tmp_matrix = Shingle_docs_RDD
#print(Shingle_docs_RDD.collect())

In [9]:
def mapper_LSH(line):
    bucket = 0
    return_list = []
    for i in range(band):
        bucket = 3*line[1][i]+11*line[1][i+1]+3
        return_list.append(((i, bucket), [line[0]]))
    return return_list

def mapper_filter(line):
    if len(line[1]) != 1:
        return [(line[0], line[1])]
    else:
        return []

In [10]:
band = 50
Shingle_docs_RDD = Shingle_docs_RDD.flatMap(mapper_LSH).reduceByKey(lambda a, b:a+b).flatMap(mapper_filter)
#print(Shingle_docs_RDD.collect())

In [11]:
def mapper_cal_sim(line):
    return_list = []
    for i in range(len(line[1])):
        for j in range(len(line[1])):
            if j >= i:
                break
            cnt = 0
            for k in range(100):
                if signature[line[1][i]-1][k] == signature[line[1][j]-1][k]:
                    cnt+=1
            return_list.append(((line[1][i], line[1][j]), cnt/100))
    return return_list

In [12]:
def mapper_adjust_KV(line):
    return line[1], (line[0][0], line[0][1])

In [13]:
tmp_matrix = tmp_matrix.take(101)
signature = [0]*101
for i in range(101):
    signature[i] = tmp_matrix[i][1]
Shingle_docs_RDD = Shingle_docs_RDD.flatMap(mapper_cal_sim).reduceByKey(lambda a, b: a).map(mapper_adjust_KV).sortByKey(ascending = False)
#print(Shingle_docs_RDD.collect())

In [14]:
output = Shingle_docs_RDD.collect()
outfile = open("Output.txt", "w")
for i in range(10):
    print("(" + str(output[i][1][0]) + ',' + str(output[i][1][1]) + "): "+ str(output[i][0]))
    outfile.write("(" + str(output[i][1][0]) + ',' + str(output[i][1][1]) + "): "+ str(output[i][0]))
    outfile.write("\n")
outfile.close()

(20,12): 1.0
(84,52): 1.0
(35,30): 0.79
(49,47): 0.72
(88,49): 0.67
(88,47): 0.5
(38,23): 0.45
(49,48): 0.36
(40,14): 0.35
(48,47): 0.25


In [15]:
sc.stop()


## 107062321 LSH Report

### mapper_split
在這個mapper裡面，我做的事情就是將每個word split開，由於有可能會split出空字串
故我先判斷這個word是不是空字串，如果不是，就再判斷他的開頭是不是數字或英文字母
如果是的話我就會把他留下

### mapper_combine3
在這個mapper裡，我做的事情就是要把一個一個word組成三個一組的3-shingle
所以我把文字第i個文字與i+1, i+2個文字中間用一個空個連起來
並且將這個string丟進一個binascii.crc32()的function裡
在這裡面他可以幫我將這組string hash成一個integer
達到講義教我們的，要把long shingle hash成4 bytes的integer

最後我再將重複出現的shingle拿掉
所以最後回傳回去的shingle set中，不會有重複的shingle
回傳後我會再用reduceByKey(lambda a, b: a+b) 將docment再次可以接上自己的全部shingle

### After shingle
將所有document的shingle拼在一起後
我會先把那個set sort過，因為後面要讓大家查index

### mapper_into_all_index
由於每個docment要知道自己這個shingle在 所有docments一起組成的shingle set中的index
等等做minhash的時候，才能得到signature
故這邊將每個docment的shingle丟進來，查找index後
組成一個 (docment, index) 的 KV pair回傳，並用flatMap攤開，並且用reduceByKey
使document各自後面就可以接著自己的shingle的index

### mapper_hash
在這個mapper裡，做的事情就是min-hash
我選擇用的100個random hash function的通式為 
    (((j+2)*line[1][i] + 3*j ) % All_shingle_len)
其中 0<=j<=99，i為document中的shingle數量 並且最後mod所有document加起來的總shingle數
故bucket為所有document的總shingle數

### mapper_LSH
在這邊要做的就是LSH，故要選擇一個hash function，足夠像的人才會被hash進同一個bucket
而我的hash function為3*line[1][i]+11*line[1][i+1]+3

其中line[1][i]與line[1][i+1]代表的是同一band中的兩個row的signature
最後回傳((第幾個band, hash到的bucket), document)
並且用reduceByKey(lambda a, b: a+b)將在同一band且被hash到同樣bucket的element放在一起

### mapper_filter
在這個mapper裡，我們要做的就是找出candidate pair
所以將若一個bucket中，只有一個element的bucket，我們要將這個bucket丟掉
也就是裡面的pair就不能成為candidate pair

### mapper_cal_sim
這個mapper就是要算candidate pair的document的similarity是多少
所以就拿出他們signature的matrix來比對
若他們某一row的signature一樣，就把一個cnt+=1
等到掃完所有的row了，就建一個新的KV pair ((doc1, doc2), similarity)
回傳後，由於可能會有重複的(doc1, doc2)，故用reduceByKey(lambda a, b: a)

### mapper_adjust_KV
最後為了sort similarity，所以我要把similarity放到key
故這個mapper只是將 similarity放到key以便sort
然後把(doc1, doc2)放到value的位置